In [2]:
import glob
import pandas as pd

import datetime

import database_info

In [3]:
engine, conn = database_info.getServerEngine()

In [6]:
files = glob.glob('../raw/uk/*')

In [16]:
files

['../raw/uk\\DemandDataUpdate.csv',
 '../raw/uk\\DemandData_2017_1551263464434.csv',
 '../raw/uk\\DemandData_2018_1551263484189.csv',
 '../raw/uk\\DemandData_2019_1581934964092.csv']

FAQ document : https://www.nationalgrideso.com/data-explorer

* Units : MW  
* Values : All values are an average over the half hour.  
* Time : All times are UK time and for the half hour beginning.  
* 

### Columns

https://data.nationalgrideso.com/demand/historic-demand-data/r/historic_demand_data_2019  
  
* ND: National Demand is the sum of metered generation, but excludes generation required to meet station load, hydro storage pumping and interconnector exports  
* FORECAST_ACTUAL_INDICATOR:Indication of whether data is out-turn (A) or forecast (F)  
* I014_ND: Equivalent to ND but calculated using settlement metered generation data from the I014 file where available 
* TSD(Transmission System Demand): Demand is equal to the ND plus the additional generation required to meet station load, pump storage pumping and interconnector exports  
* I014_TSD: Equivalent to TSD (above), but calculated using settlement metered generation data from the I014 file where available  
  
  
* settlement metered generation:
* station load: power that generators themselves require to generate electricity.(소내전력)

In [52]:
df = pd.DataFrame()

for file in files:
    tmp = pd.read_csv(file)
    df = pd.concat([df, tmp], ignore_index=True)

In [54]:
df.isna().sum()

SETTLEMENT_DATE                  0
SETTLEMENT_PERIOD                0
ND                               0
FORECAST_ACTUAL_INDICATOR    52560
I014_ND                          0
TSD                              0
I014_TSD                         0
ENGLAND_WALES_DEMAND             0
EMBEDDED_WIND_GENERATION         0
EMBEDDED_WIND_CAPACITY           0
EMBEDDED_SOLAR_GENERATION        0
EMBEDDED_SOLAR_CAPACITY          0
NON_BM_STOR                      0
PUMP_STORAGE_PUMPING             0
I014_PUMP_STORAGE_PUMPING        0
FRENCH_FLOW                      0
BRITNED_FLOW                     0
MOYLE_FLOW                       0
EAST_WEST_FLOW                   0
NEMO_FLOW                    35040
I014_FRENCH_FLOW                 0
I014_BRITNED_FLOW                0
I014_MOYLE_FLOW                  0
I014_EAST_WEST_FLOW              0
dtype: int64

In [55]:
df['LoadDay'] = df['SETTLEMENT_DATE'].apply(lambda x: datetime.datetime.strptime(x, '%d-%b-%Y'))+\
df['SETTLEMENT_PERIOD'].apply(lambda x:datetime.timedelta(minutes=30*x))

In [56]:
df['LoadDay']

0       2020-04-01 00:30:00
1       2020-04-01 01:00:00
2       2020-04-01 01:30:00
3       2020-04-01 02:00:00
4       2020-04-01 02:30:00
                ...        
55531   2019-12-31 22:00:00
55532   2019-12-31 22:30:00
55533   2019-12-31 23:00:00
55534   2019-12-31 23:30:00
55535   2020-01-01 00:00:00
Name: LoadDay, Length: 55536, dtype: datetime64[ns]

In [57]:
df = df.sort_values(by='LoadDay')
df = df.reset_index(drop=True)

In [58]:
df

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,FORECAST_ACTUAL_INDICATOR,I014_ND,TSD,I014_TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,...,FRENCH_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,I014_FRENCH_FLOW,I014_BRITNED_FLOW,I014_MOYLE_FLOW,I014_EAST_WEST_FLOW,LoadDay
0,01-JAN-2017,1,27316,NaN,27117,28013,27833,24789,1514,4871,...,923,857,295,233,NaN,947,857,296,240,2017-01-01 00:30:00
1,01-JAN-2017,2,27900,NaN,27706,28539,28329,25486,1518,4871,...,997,874,297,287,NaN,986,874,297,286,2017-01-01 01:00:00
2,01-JAN-2017,3,27588,NaN,27286,28279,27916,25254,1543,4871,...,997,818,297,287,NaN,986,816,297,286,2017-01-01 01:30:00
3,01-JAN-2017,4,26456,NaN,26169,27819,27550,24270,1567,4871,...,997,822,297,287,NaN,986,828,297,287,2017-01-01 02:00:00
4,01-JAN-2017,5,25429,NaN,25150,27565,27316,23409,1601,4871,...,997,854,297,281,NaN,986,855,297,281,2017-01-01 02:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55531,01-JUN-2020,44,0,F,0,0,0,0,976,6527,...,0,0,0,0,0.0,0,0,0,0,2020-06-01 22:00:00
55532,01-JUN-2020,45,0,F,0,0,0,0,940,6527,...,0,0,0,0,0.0,0,0,0,0,2020-06-01 22:30:00
55533,01-JUN-2020,46,0,F,0,0,0,0,904,6527,...,0,0,0,0,0.0,0,0,0,0,2020-06-01 23:00:00
55534,01-JUN-2020,47,0,F,0,0,0,0,871,6527,...,0,0,0,0,0.0,0,0,0,0,2020-06-01 23:30:00


In [59]:
df = df[df['FORECAST_ACTUAL_INDICATOR']!='F']
df = df.reset_index(drop=True)

In [61]:
df

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,FORECAST_ACTUAL_INDICATOR,I014_ND,TSD,I014_TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,...,FRENCH_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,I014_FRENCH_FLOW,I014_BRITNED_FLOW,I014_MOYLE_FLOW,I014_EAST_WEST_FLOW,LoadDay
0,01-JAN-2017,1,27316,NaN,27117,28013,27833,24789,1514,4871,...,923,857,295,233,NaN,947,857,296,240,2017-01-01 00:30:00
1,01-JAN-2017,2,27900,NaN,27706,28539,28329,25486,1518,4871,...,997,874,297,287,NaN,986,874,297,286,2017-01-01 01:00:00
2,01-JAN-2017,3,27588,NaN,27286,28279,27916,25254,1543,4871,...,997,818,297,287,NaN,986,816,297,286,2017-01-01 01:30:00
3,01-JAN-2017,4,26456,NaN,26169,27819,27550,24270,1567,4871,...,997,822,297,287,NaN,986,828,297,287,2017-01-01 02:00:00
4,01-JAN-2017,5,25429,NaN,25150,27565,27316,23409,1601,4871,...,997,854,297,281,NaN,986,855,297,281,2017-01-01 02:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55164,25-MAY-2020,13,17456,A,0,18426,0,15803,605,6527,...,-370,433,402,504,639.0,0,0,0,0,2020-05-25 06:30:00
55165,25-MAY-2020,14,17459,A,0,18485,0,15754,584,6527,...,-372,440,383,459,639.0,0,0,0,0,2020-05-25 07:00:00
55166,25-MAY-2020,15,17744,A,0,18409,0,15895,593,6527,...,13,721,272,325,686.0,0,0,0,0,2020-05-25 07:30:00
55167,25-MAY-2020,16,18066,A,0,18581,0,16122,601,6527,...,27,733,331,333,687.0,0,0,0,0,2020-05-25 08:00:00


In [62]:
df.isna().sum()

SETTLEMENT_DATE                  0
SETTLEMENT_PERIOD                0
ND                               0
FORECAST_ACTUAL_INDICATOR    52560
I014_ND                          0
TSD                              0
I014_TSD                         0
ENGLAND_WALES_DEMAND             0
EMBEDDED_WIND_GENERATION         0
EMBEDDED_WIND_CAPACITY           0
EMBEDDED_SOLAR_GENERATION        0
EMBEDDED_SOLAR_CAPACITY          0
NON_BM_STOR                      0
PUMP_STORAGE_PUMPING             0
I014_PUMP_STORAGE_PUMPING        0
FRENCH_FLOW                      0
BRITNED_FLOW                     0
MOYLE_FLOW                       0
EAST_WEST_FLOW                   0
NEMO_FLOW                    35040
I014_FRENCH_FLOW                 0
I014_BRITNED_FLOW                0
I014_MOYLE_FLOW                  0
I014_EAST_WEST_FLOW              0
LoadDay                          0
dtype: int64

In [63]:
ndf = df[['LoadDay','ND',]]

In [64]:
ndf

,LoadDay,ND
0,2017-01-01 00:30:00,27316
1,2017-01-01 01:00:00,27900
2,2017-01-01 01:30:00,27588
3,2017-01-01 02:00:00,26456
4,2017-01-01 02:30:00,25429
...,...,...
55164,2020-05-25 06:30:00,17456
55165,2020-05-25 07:00:00,17459
55166,2020-05-25 07:30:00,17744
55167,2020-05-25 08:00:00,18066


In [66]:
ndf.to_sql('uk_ngeso', conn)